In [5]:
import cv2
import numpy as np
import csv
from utils import *

def kalman_filter_tracking(A, B, output_csv="output.csv"):
    n = 3  # Số lượng khung giống nhất
    P = np.eye(n) * 1000  # Covariance Matrix of State Estimate (đại diện cho độ không chắc chắn trạng thái ban đầu)
    F = np.eye(n)  # State transition matrix (ma trận chuyển trạng thái n x n)
    Q = np.eye(n) * 0.1  # Process noise covariance (nhiễu quá trình, biểu thị sự không chắc chắn trong mô hình)
    R = np.eye(n) * 10   # Measurement noise covariance (nhiễu đo lường, biểu thị sự không chắc chắn trong các phép đo)
    
    # Ghi kết quả vào file .csv
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frame_B", "Match_1", "Match_2", "Match_3"])  # Header
        
        for i in range(len(B)):
            if i == 0:
                scores = []
                img1 = cv2.cvtColor(B[0], cv2.COLOR_BGR2GRAY)

                for j, img_file in enumerate(A):
                    img2 = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
                    similarity_percentage = calculate_similarity_percentage_akaze(img1, img2)
                    scores.append(similarity_percentage)
            
                sorted_indices = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
                top_3_values = sorted_indices[:3]
                state = np.array([item[0] for item in top_3_values])
                print(f"Initial state: {state}")
                
            else:
                # Dự đoán
                state_pred = F @ state # Chuyển trạng thái (trường hợp này ko đổi vì F là ma trận đơn vị)
                P_pred = F @ P @ F.T + Q # Tính hiệp phương sai
                
                # Đo lường
                img_b = cv2.cvtColor(B[i], cv2.COLOR_BGR2GRAY)
                scores = [calculate_similarity_percentage_orb(img_b, cv2.cvtColor(A[j], cv2.COLOR_BGR2GRAY)) for j in range(len(A))]
                top_indices = np.argsort(scores)[-n:]  # Lấy n điểm có giá trị cao nhất

                # # Loại bỏ các giá trị trùng lặp
                # top_indices = np.unique(top_indices)
                # if len(top_indices) < n:
                #     sorted_indices = np.argsort(scores)
                #     for idx in sorted_indices[::-1]:
                #         if idx not in top_indices:
                #             top_indices = np.append(top_indices, idx)
                #         if len(top_indices) == n:
                #             break

                # Cập nhập tham số bộ lọc Kalman
                z = np.array(top_indices)
                y = z - state_pred  # Sai số đo lường
                S = P_pred + R  # Hiệp phương sai của sai số đo
                K = P_pred @ np.linalg.inv(S)  # Kalman gain
                
                # Cập nhật ước tính trạng thái và hiệp phương sai
                state = state_pred + K @ y
                P = (np.eye(n) - K) @ P_pred
                
                # Loại bỏ giá trị trùng lặp của Kalman
                state = np.unique(state.astype(int))
                if len(state) < n:
                    for idx in z:
                        if idx not in state:
                            state = np.append(state, idx)
                        if len(state) == n:
                            break
                
                # Xuất kết quả
                print(f"Frame B[{i}] matches best with frames {state} in A")
                
            # Viết vào file .csv
            writer.writerow([i] + state.tolist())

In [4]:
import os 

compare_path = "D:/Xu ly anh/thucnghiem/v47"
B1 = os.listdir(compare_path)
B1.sort()
B = [cv2.imread(os.path.join(compare_path, img_file)) for img_file in B1]
folder_path = "D:/Xu ly anh/thucnghiem/v44"
A1 = os.listdir(folder_path)
A1.sort()
A = [cv2.imread(os.path.join(folder_path, img_file)) for img_file in A1]

#A là frame của ảnh pano gốc
#B là các frame cần tracking


In [6]:
kalman_filter_tracking(A, B, output_csv="tracking_output.csv")

Initial state: [700 702 705]
Frame B[1] matches best with frames [699 702 707] in A
Frame B[2] matches best with frames [701 705 709] in A
Frame B[3] matches best with frames [701 705 712] in A
Frame B[4] matches best with frames [701 704 710] in A
Frame B[5] matches best with frames [702 706 712] in A
Frame B[6] matches best with frames [701 706 711] in A
Frame B[7] matches best with frames [701 708 712] in A
Frame B[8] matches best with frames [702 709 712] in A
Frame B[9] matches best with frames [701 710 713] in A
Frame B[10] matches best with frames [701 709 714] in A
Frame B[11] matches best with frames [701 708 714] in A
Frame B[12] matches best with frames [701 707 713] in A
Frame B[13] matches best with frames [699 706 712] in A
Frame B[14] matches best with frames [697 704 710] in A
Frame B[15] matches best with frames [690 701 707] in A
Frame B[16] matches best with frames [627 695 706] in A
Frame B[17] matches best with frames [632 694 705] in A
Frame B[18] matches best wit

In [17]:
import pandas as pd

# Đọc file .csv
df = pd.read_csv('tracking_output.csv')

# Tạo danh sách các mảng, với mỗi phần tử là một danh sách các giá trị tương ứng
compare = df[['Match_1', 'Match_2', 'Match_3']].values.tolist()

In [10]:
output_folder = "D:/Xu ly anh/thucnghiem/v44_compare_kalman_1"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [62]:
csv_file = "D:/Xu ly anh/thucnghiem/homographies_DJI_0044_sift.csv"
H = read_homographies_from_csv(csv_file)

output_width, output_height, min_x, min_y = calculate_output_size(A, H)
output_shape = (output_width, output_height)
result_origin = cv2.imread("D:/Xu ly anh/thucnghiem/result_DJI_0044_sift.jpg")
print(output_shape)


(3082, 5851)


In [54]:
compare_color = []
for i in range(len(compare)):
    top_3_values = compare[i]
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    top_3_values = [(item, colors[i]) for i, item in enumerate(top_3_values)]
    compare_color.append(top_3_values)



In [69]:
for index, position in enumerate(compare_color):
    result = copy.deepcopy(result_origin)
    for pos, color in position:
        img = A[pos]
        img = draw_outer_bounding_box(img, color=color)
        warped_image = warp_image_to_zero(img, H, pos, output_shape, min_x, min_y)
        result = np.maximum(result, warped_image)

        frame_path = os.path.join(output_folder, f"frame_{index:04d}_compare.jpg")
        result_compare = concatenate_and_label_images(result, B[index], "result_compare", f"frame_{index:04d}.jpg")
        cv2.imwrite(frame_path, result_compare)
